## Схема проекта

**Принципиальные моменты:**
*   У страховой компании нет времени "на подумать", решение о приеме на страхование или отказе, а также о тарифе должено быть сделано непосредственно в момент обращения клиента за полисом. Соответственно, сервис должен работать в режиме реального времени и давать ответ в течение максимум нескольких секунд.
*   Сервис скоринга строится обычно отдельным блоком, а фронт-система взаимодействует с сервисом через API (Application programming interface).
*   Запрос данных из внешних источников может производиться как на уровне фронт-системы, так и на уровне сервиса.
*   Один из возможных вариантов - Flask.
*   Передача информации производится путем обмена json-файлами.
*   Для тестирования можно использовать Postman.

  ![](https://drive.google.com/uc?export=view&id=1OAOF1M2U14UJWDmeJg2mwo-pgSwyNyzc)

## JSON

JSON (JavaScript Object Notation) - простой текстовый формат обмена данными, он основан на подмножестве языка программирования JavaScript.<br/>
Например, строка из нашего датасета выглядела бы следующим образом:<br/>
```
{
"ID": 1,
"Exposure": 0.583,
"RecordBeg": "2004-06-01",
"RecordEnd": "",
"DrivAge": 55,
"Gender": "Female",
...
}
```

## При внедрении

**При внедрении необходимо сделать:**
*   Определить формат json'а, в котором данные будут приниматься сервисом и отправляться обратно.
*   Определить ip-адрес и порт, на который будут поступать данные.
*   Создать во Flask необходимые роуты:<br/>
    `@app.route('/predict_example', method='POST')`<br/>
    `def predict_example():`
*   Перенести во Flask все функции преобразования данных,
    *   формат данные, приходящих от фронт-системы, может отличаться от формата исторических данных, использовавшихся при построении модели; в результате преобразований данные на вход модели должны поступить ровно в том виде, в каком была обучена модель.
*   Загрузить обученные модели.
*   Настроить логирование, запись котировок.

**Особенности:**
*   Библиотека H2O использует виртуальную Java-машину:
    *   ее нужно инициализировать один раз, а не поднимать заново для каждого расчета;
    *   вручную выделить под нее отдельный порт и указать размер используемой памяти;
    *   не создавать каждый раз заново H2O-Frame, а записывать в единожды подготовленный.
*   Необходимо удостовериться, что на всех этапах сервис отрабатывает корректно; например, можно иметь заготовленный массив котировок с заранее известными ответами.
*   Необходимо провести нагрузочное тестирование и удостовериться, что сервис справляется с нагрузкой.
*   Для согласованности версий Python, Java при переносе на другие серверы имеет смысл использовать докеры.



## Flask

Тут будет сервис для обработки запросов на Flask

In [ ]:
# Подключение к Google drive

# from google.colab import drive
# drive.mount('/content/drive')

Google Colab предоставляет виртуальную машину, поэтому мы не можем получить доступ к локальному хосту, как это делаем на нашем локальном компьютере при запуске локального веб-сервера. Что мы можем сделать, так это предоставить его общедоступному URL-адресу с помощью ngrok.

https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b

In [ ]:
# !pip install flask-ngrok
# !pip install flask

In [1]:
# from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

In [11]:
# Пробный запуск Flask # надо выключить!

app = Flask(__name__)
# run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/a")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Dec/2021 07:57:55] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [04/Dec/2021 07:58:04] "GET /a HTTP/1.1" 200 -


In [3]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve,scorer
import dill
dill._dill._reverse_typemap['ClassType'] = type

/home/morrowbord/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


### **Создаем сервис для обработки запросов к модели**

In [4]:
# path = '/content/gdrive/MyDrive/2_ver_actual Артем Зраев. 10.2020/Lection9/'
path2='/home/morrowbord/jupiyer-notebook/ML_business/project_churn/'

In [5]:
# Загружаем обученные модели
with open(path2 + 'model_RF.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [6]:
X_test = pd.read_csv(path2+"X_test.csv")
y_test = pd.read_csv(path2+"y_test.csv")

Запустить сервис и не глушить его, пока работаем 

In [ ]:
# Обработчики и запуск Flask
app = Flask(__name__)
# run_with_ngrok(app)  # Start ngrok when app is run

@app.route('/predict', methods=['GET', 'POST'])
def predict():
    data = {"success": False}
  # ensure an image was properly uploaded to our endpoint
    if request.method == "POST":
#       description, company_profile, benefits = "", "", "qq"
        gender, tenure, PhoneService, TotalCharges, StreamingMovies, StreamingTV, TechSupport = "","","","","","","qq"
        request_json = request.get_json()
    if request_json["gender"]:
        description = request_json['gender']
  
    if request_json["tenure"]:
        company_profile = request_json['tenure']
		       
    if request_json["PhoneService"]:
        benefits = request_json['PhoneService']
    print(description )  
    preds = model.predict_proba(pd.DataFrame({"gender": [gender],
                                            "tenure": [tenure],
                                            "PhoneService": [PhoneService],
                                           "TotalCharges": [TotalCharges],
                                            "StreamingMovies":[StreamingMovies], 
                                            "StreamingTV":[StreamingTV],
                                            "TechSupport":[TechSupport]
                                            
                                           }))
    data["predictions"] = preds[:, 1][0]
    data["description"] = description
	# indicate that the request was a success
    data["success"] = True

	# return the data dictionary as a JSON response
    return jsonify(data)
 

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2021-12-04 07:59:13,292] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "/home/morrowbord/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/morrowbord/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/morrowbord/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/morrowbord/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/morrowbord/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/morrowbord/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return se

['gender']
